In [79]:
import time

import requests
import random


In [80]:
wallets = [ ("3059301306072a8648ce3d020106082a8648ce3d03010703420004989296a03a8912d634eb7375244651572c35e708f2980fa03f5e9561b6ab7c07ff5e69eabc01a5c52ecd5f1a2b3ec0607fe46b4d4ef57fa536515f78723546f5", "308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b02010104202f77dfa14fdb585d90a4e078e70584aae536bcdb57c9f9d81e77c0dc663ed748a14403420004989296a03a8912d634eb7375244651572c35e708f2980fa03f5e9561b6ab7c07ff5e69eabc01a5c52ecd5f1a2b3ec0607fe46b4d4ef57fa536515f78723546f5"),
("3059301306072a8648ce3d020106082a8648ce3d03010703420004e4ddc7058cd86915d679aa464710e7407cb623689a07bb77340b94dd043e27c47b8f105f095f177bd52643afe4aee8e63f636ea11308ee265bf169acd3f595ae", "308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b02010104200b4231c99eea60d6742278485903e21c4a1763426a6ba73a880da071beb6d35aa14403420004e4ddc7058cd86915d679aa464710e7407cb623689a07bb77340b94dd043e27c47b8f105f095f177bd52643afe4aee8e63f636ea11308ee265bf169acd3f595ae"),
("3059301306072a8648ce3d020106082a8648ce3d03010703420004ee7455a9894266fdbc23b6afc39d894a71a837bad62aa5e6cd63ae7a9d1d1e795645bb9e612ff2ae9c5db1775e605a02016ccaf48860f821bd42841702825317", "308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b02010104208399d6745eea61b3e55bca3349fa0c4abc5669a351e7b4d18bd89f55668df9cfa14403420004ee7455a9894266fdbc23b6afc39d894a71a837bad62aa5e6cd63ae7a9d1d1e795645bb9e612ff2ae9c5db1775e605a02016ccaf48860f821bd42841702825317"),
]

In [81]:
def generate_transaction(sender: str, sender_private_key: str, receiver: str, amount: str):
  return requests.post("http://localhost:3001/generate/transaction",
                       {
                         "sender_address": sender,
                         "sender_private_key": sender_private_key,
                         "receiver_address": receiver,
                         "amount": amount
                       }).json()

In [82]:
def get_arbitrary_transaction():
  max_index = len(wallets) - 1
  random_sender_index = random.randint(0, max_index)
  random_receiver_index = random.randint(0, max_index)

  while random_receiver_index == random_sender_index:
    random_receiver_index = random.randint(0, max_index)

  sender = wallets[random_sender_index]
  sender_address = sender[0]
  sender_private_key = sender[1]
  receiver_address = wallets[random_receiver_index][0]
  amount = str(random.randint(1, 1000)),

  transaction = generate_transaction(sender_address, sender_private_key, receiver_address, amount)

  return {
    "amount": transaction["transaction"]["amount"],
    "sender": sender_address,
    "receiver": receiver_address,
    "signature": transaction["signature"],
    "timestamp": transaction["transaction"]["timestamp"],
  }

In [83]:
def get_block(index, timestamp, nonce, previous_hash, transactions):
  return {
    "index": index,
    "timestamp": timestamp,
    "nonce": nonce,
    "previous_hash": previous_hash,
    "transactions": transactions,
  }

In [84]:
def send_chain():
  endpoint = "http://localhost:2001/chain"

  transactions = list()
  for i in range(10):
    transactions.append(get_arbitrary_transaction())

  block0 = get_block(index=0,
                    timestamp=str(time.time()),
                    previous_hash="",
                    nonce=0,
                    transactions=transactions[:5],
                    )
  block1 = get_block(index=1,
                    timestamp=str(time.time()),
                    previous_hash="",
                    nonce=1,
                    transactions=transactions[5:],
                    )

  res = requests.post(endpoint, json={
    "chain": {
      "blocks": [block0, block1]
    }
  })

  return res

In [85]:
def get_chain():
  endpoint = "http://localhost:2001/chain"

  res = requests.get(endpoint)
  return res.json()

In [86]:
res = send_chain()
print(res)

<Response [204]>


In [87]:
res = get_chain()
print(res)

{'chain': {'blocks': [{'index': 0, 'nonce': 0, 'previous_hash': '', 'timestamp': '1675805484.927965', 'transactions': [{'amount': '218', 'receiver': '3059301306072a8648ce3d020106082a8648ce3d03010703420004e4ddc7058cd86915d679aa464710e7407cb623689a07bb77340b94dd043e27c47b8f105f095f177bd52643afe4aee8e63f636ea11308ee265bf169acd3f595ae', 'sender': '3059301306072a8648ce3d020106082a8648ce3d03010703420004ee7455a9894266fdbc23b6afc39d894a71a837bad62aa5e6cd63ae7a9d1d1e795645bb9e612ff2ae9c5db1775e605a02016ccaf48860f821bd42841702825317', 'signature': '3ce312ad948fc5c078484cfb13df7296d1b0f4880533fbe68eb3ea21f97c9ba10e5d7a4cae807e79cd9bca7290bda6500348f0e82b48920ff553bfb0550d0540', 'timestamp': '1675805484.764616'}, {'amount': '245', 'receiver': '3059301306072a8648ce3d020106082a8648ce3d03010703420004ee7455a9894266fdbc23b6afc39d894a71a837bad62aa5e6cd63ae7a9d1d1e795645bb9e612ff2ae9c5db1775e605a02016ccaf48860f821bd42841702825317', 'sender': '3059301306072a8648ce3d020106082a8648ce3d03010703420004989296a0